In [ ]:
import streamlit as st
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, Wav2Vec2Processor, Wav2Vec2ForCTC
import soundfile as sf
from diffusers import StableDiffusionPipeline
import tempfile
import os

device = 0 if torch.cuda.is_available() else -1
emotion_model_name = "j-hartmann/emotion-english-distilroberta-base"
emotion_classifier = pipeline("text-classification", model=emotion_model_name, return_all_scores=False, device=device)

asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

sd_pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
sd_pipe.to("cuda" if torch.cuda.is_available() else "cpu")

st.title("Emotion Painter (AI)")

input_type = st.radio("Select input type", ["Text", "Audio"])
user_text = ""
if input_type == "Text":
    user_text = st.text_area("Enter your sentence expressing your feeling:")
else:
    audio_file = st.file_uploader("Upload audio file (wav)", type=["wav"])
    if audio_file:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
            tmp.write(audio_file.read())
            tmp_path = tmp.name
        speech, rate = sf.read(tmp_path)
        input_values = asr_processor(speech, sampling_rate=rate, return_tensors="pt").input_values
        logits = asr_model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        user_text = asr_processor.batch_decode(predicted_ids)[0]

if st.button("Generate Image") and user_text.strip():
    emotion_result = emotion_classifier(user_text)[0]
    emotion = emotion_result['label']
    prompt = f"An artistic representation of {emotion} emotion"
    image = sd_pipe(prompt).images[0]
    st.image(image, caption=f"Emotion: {emotion}")
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as img_tmp:
        image.save(img_tmp.name)
        st.download_button("Download Image", open(img_tmp.name, "rb"), file_name="emotion_art.png")